In [ ]:
# default_exp ads

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore

     |████████████████████████████████| 61kB 2.6MB/s 


In [ ]:
#hide
!pip install -qU gspread facebook_business python-facebook-api

     |████████████████████████████████| 1.0MB 6.8MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 10.1MB 14.5MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 


In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

## Facebook Auth

In [ ]:
#hide
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env AD_ACC_ID {data.get("AD_ACC_ID")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
from tomorrowswig_followers.core import *

In [ ]:
#export
import os
from typing import *

from datetime import datetime, timedelta
from functools import partial

import pandas as pd
import numpy as np
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adaccount import AdAccount

AD_ACC_ID = os.environ.get("AD_ACC_ID")

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Ads reporting

In [ ]:
#export
FacebookAdsApi.init(APP_ID, APP_SECRET, TOKEN)
account = AdAccount(AD_ACC_ID)

In [ ]:
datetime.fromtimestamp(1595946718)

datetime.datetime(2020, 7, 28, 14, 31, 58)

In [ ]:
datetime(2020,8,10,23,59).strftime('%s')

'1597103940'

https://github.com/facebook/facebook-python-business-sdk/blob/master/facebook_business/adobjects/adsinsights.py

In [ ]:
insights = account.get_insights(fields=[
    "ad_id",
    "ad_name",
    "clicks",
    "cpc",
    "ctr",
    "cpm",
    "cost_per_action_type",
    "spend",
    "impressions",
    "reach",
    "actions",
    "video_avg_time_watched_actions",
    "video_p50_watched_actions",
    "video_p75_watched_actions",
    "video_p95_watched_actions",

], params={
    'level': "ad",
    'date_preset': "yesterday",
    # "date_start": "1595116800",
    # "date_stop": "1595203200"
})


In [ ]:
from facebook_business.adobjects.adsinsights import *
fields = [f for f in AdsInsights.Field.__dict__ if not f.startswith("__")]

In [ ]:
" ".join([fields[56], fields[105]])

'cost_per_unique_conversion unique_conversions'

In [ ]:
# account.get_insights(fields=["ad_id", "quality_ranking", "engagement_rate_ranking",
#                              "conversion_rate_ranking", "ad_name"], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.yesterday,
# })

In [ ]:
# account.get_insights(fields=fields[:56] + fields[57:105] + fields[106:], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.last_3d,
# })[2]

In [ ]:
#export
COLUMNS = ["Ad Name", "Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow",
           "Clicks (All)", "Link Clicks", "CPC (All)" , "CPC (Cost per Link Click)",
           "CTR (All)", "CPM (Cost per 1,000 Impressions)", "Amount Spent (USD)", "Impressions", "Reach", "Post Reactions", "Post Shares",
           "Video Average Play Time", "Video Plays at 50%", "Video Plays at 75%", "Video Plays at 95%"]

In [ ]:
#export
def get_insights(date: str) -> Dict:
    return account.get_insights(fields=[
        "ad_id",
        "ad_name",
        "clicks",
        "cpc",
        "ctr",
        "cpm",
        "cost_per_action_type",
        "spend",
        "impressions",
        "reach",
        "actions",
        "video_avg_time_watched_actions",
        "video_p50_watched_actions",
        "video_p75_watched_actions",
        "video_p95_watched_actions",
    ], params={
        "level": "ad",
        "time_range": {"since": date,
                       "until": date},
    })

In [ ]:
#export
def get_action(cell, name):
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return float(action[0]["value"])
    return 0

In [ ]:
#export
def add_country_total(df: pd.DataFrame) -> Tuple[pd.DataFrame, str]:
    dupes = df[df.index.duplicated(False)]
    if not len(dupes):
        return

    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)"]
    to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
    total = dupes.iloc[0,:].copy()
    index = dupes.index[0]
    total["ad_name"] = f"{index} total"
    total[to_mean] = dupes[to_mean].apply(np.mean)
    total[to_sum] = dupes[to_sum].astype(int).apply(sum)
    return df.append(total), index

In [ ]:
ad = Ad("23844695610150691")

In [ ]:
ad = ad.api_get(fields=["targeting"])
ad["targeting"]["geo_locations"]["countries"][0]

'CA'

In [ ]:
#export
def get_countries(ids: List[str]) -> List[Dict]:
    countries = []
    for id in ids:
        ad = Ad(id).api_get(fields=["targeting"])
        country = ad["targeting"]["geo_locations"]["countries"][0]
        countries.append({"ad_id": id, "country": country})
    return countries

In [ ]:
assert get_countries(["23844695610150691", "23844698190420691"]) == [{'ad_id': '23844695610150691', 'country': 'CA'},
                                                                     {'ad_id': '23844698190420691', 'country': 'US'}]

In [ ]:
def create_first_part(date:str ):
    df = pd.DataFrame(get_insights(date))
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)

    df = df.merge(pd.DataFrame(get_countries(df["ad_id"].values)))
    df.set_index(df["country"], inplace=True)
    df.sort_values("ad_name", inplace=True)

    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click"))
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions", "ad_id", "country"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    dupes = add_country_total(df)
    return df.round(2)

In [ ]:
df = create_first_part("2020-06-10")

In [ ]:
to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time"]
empty = ["Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow", "ad_name"]
to_sum = df.columns.difference(to_mean + empty)
total = pd.Series(name="total", index=df.columns, dtype=float)

In [ ]:
total[to_mean] = df[to_mean].apply(np.mean)
total[to_sum] = df[to_sum].apply(sum)

In [ ]:
df = df.append(total)

In [ ]:
#export
def add_total(df: pd.DataFrame) -> pd.DataFrame:
    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time"]
    empty = ["Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow", "ad_name"]
    to_sum = df.columns.difference(to_mean + empty)
    total = pd.Series(name="total", index=df.columns, dtype=float)
    total[to_mean] = df[to_mean].apply(np.mean)
    total[to_sum] = df[to_sum].apply(sum)
    df = df.append(total)
    return pd.concat([df.iloc[-1:], df.iloc[:-1]])

In [ ]:
#export
def get_insights_df(insights: List) -> pd.DataFrame:
    df = pd.DataFrame(insights)
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)

    df = df.merge(pd.DataFrame(get_countries(df["ad_id"].values)))
    df.set_index(df["country"], inplace=True)
    df.sort_values("ad_name", inplace=True)

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click"))
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions", "ad_id", "country"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    dupes = add_country_total(df)
    df = add_total(df)
    df.rename(columns={"ad_name": "Ad Name", "clicks": "Clicks (All)", "cpc": "CPC (All)",
                       "ctr": "CTR (All)", "cpm": "CPM (Cost per 1,000 Impressions)",
                       "reach": "Reach", "impressions": "Impressions", "spend": "Amount Spent (USD)"}, inplace=True)
    df["Ads Followers Change"] = 0
    df["% (Clicks)"] = 0
    df["% (Impressions)"] = 0
    df["Cost Per Follow"] = 0
    return df[COLUMNS].round(2)

In [ ]:
#export
def create_insights(event: Dict = None, context=None,) -> str:
    yesterday = datetime.today() - timedelta(days=1)
    insights = get_insights(yesterday.strftime("%Y-%m-%d"))
    df = get_insights_df(insights)
    df_size = len(df)

    empty = pd.Series(name="", dtype=str)
    df = df.append([empty] * 25)

    worksheet_name = yesterday.strftime("%b %d %Y")
    write_df(df, worksheet_name)
    notes_df = pd.DataFrame([["", "CONSIDER:", "", "", "TO DO:"]], index=["PERFORMANCE:"], columns=[""] * 5)
    write_df(notes_df, worksheet_name, loc=f"A{df_size+10}")

    wsh = get_worksheet(worksheet_name)
    wsh.format(f"C1:F{df_size+1}", {"textFormat": {
        "foregroundColor": {
            "red": 0.45,
            "green": 0.0,
            "blue": 0.0
        }}},)
    wsh.format(f"D:E", {"numberFormat": {"type": "PERCENT"}},)
    wsh.format(f"F", {"numberFormat": {"type": "CURRENCY"}},)
    return f"Created insights in '{worksheet_name}'"

In [ ]:
create_insights()

"Created insights in 'Aug 13 2020'"

In [ ]:
# DONE estimate, real

# TODO

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.


## Guesses
Crowdfunding cost of a follower in Germany

1. Why base on crowdfunding? It's to raise money to create a product, not to sell a product. Crowdfunding is 90-99% corporate - which is noise.
2. Two questions - how to grow followers in instagram and how to advertise a book.

How to advertise a book:
1. Germany (spending power, etc) https://www.statista.com/outlook/335/137/crowdfunding/germany#market-arpu
2. Books worldwide https://www.statista.com/outlook/21300/100/books/worldwide#market-globalRevenue

How to grow instagram
https://buffer.com/library/instagram-growth/
https://www.shopify.com/blog/14288561-how-to-build-a-massive-following-on-instagram